# <span style="color: blue;">Менеджеры контекста</span>

In [23]:
def do_something(*args, **kwargs): 
    pass

### Зачем нужны менеджеры контекста?

Менеджеры контекста позволяют компактно выразить уже знакомый нам паттерн управления ресурсами _(файл, сокет, мьютекс, соединение с БД, транзакция и т.п.)_:


In [ ]:
r = acquire_resource()  # завладели ресурсом
try:
    do_something(r)
finally:
    release_resource(r)  # освобождение ресурса

С помощью менеджера контекста пример выше можно записать так:

In [ ]:
with acquire_resource() as r:
    do_something(r)

Действие `release_resource` будет выполнено автоматически, вызывать его явно не нужно.

Интерпретатор не знает, как освобождать ресурсы, но каждый ресурс знает, как себя освободить.

Оператор **`as`** опционален.

### Использование переменных

In [ ]:
f = 'hi'

with open('hello.txt') as f:
    pass

print(f)

for i in range(4):
    pass

print(i)

### Протокол менеджеров контекста

Протокол менеджеров контекста состоит из двух методов.

Метод **`__enter__`** инициализирует контекст _(например, открывает файл или захватывает мьютекс)_

Значение, возвращаемое методом **`__enter__`**, записывается по имени, указанному после оператора **`as`**.

Метод **`__exit__`** вызывается после выполнения тела оператора **`with`**. 

Метод принимает три аргумента:
1. тип исключения
2. само исключение
3. объект типа `traceback`

Если в процессе исполнения тела оператора **`with`** было поднятно исключение, метод **`__exit__`** может подавить его, вернув **`True`**.

Если исключения не было, все три аргумента равняются `None`

Экземпляр любого класса, реализующего эти два метода, является **менеджером контекста**<br/>
_(то есть наследоваться для этого не нужно)_

Такая схема называется **Duck Typing** _("если оно крякает как утка, значит, оно и есть утка")_

### “Семантика” оператора with

Напоминание:

In [ ]:
with acquire_resource() as r:
    do_something(r)

Процесс исполнения оператора `with` можно концептуально записать так:

In [ ]:
manager = acquire_resource()
r = manager.__enter__()
try:
    do_something(r)
finally:
    exc_type, exc_value, tb = sys.exc_info()
    suppress = manager.__exit__(exc_type, exc_value, tb)
    if exc_value is not None and not suppress:
        raise exc_value

### Расширенные возможности оператора with

Оператор `with` позволяет работать с несколькими контекстными менеджерами одновременно:


In [ ]:
with acquire_resource() as r, \
     acquire_other_resource() as other:
    do_something(r, other)

Здесь обязательно использовать именно слеш для разбития строки на две (скобочки не помогут)

Такая запись эквивалентна двум вложенным менеджерам контекста:

In [ ]:
with acquire_resource() as r:
    with acquire_other_resource() as other:
        do_something(r, other)

И раскручиваться они будут в обратном порядке

Можно также использовать оператор `with` без указания имени переменной:

In [ ]:
with acquire_resource():
    do_something()

### Примеры менеджеров контекста: opened

In [ ]:
from functools import partial
class opened:
    def __init__(self, path, *args, **kwargs):
        self.opener = partial(open, path, *args, **kwargs)

    def __enter__(self):
        self.handle = self.opener()
        return self.handle

    def __exit__(self, *exc_info):
        self.handle.close()
        del self.handle
        # Почему можно обойтись без return?

with opened("./example.txt", mode="rt") as handle:
    pass

**Капитан сообщает:** 

`opened` интересен только в качестве примера, потому что файлы в Python уже поддерживают протокол менеджеров контекста.

### Примеры менеджеров контекста: модуль tempfile

Модуль `tempfile` реализует классы для работы с временными файлами.

Все классы реализуют протокол менеджеров контекста, которые работают так же, как и для обычных файлов.

Интересный пример — класс `TemporaryFile`, который автоматически удаляет временный файл при выходе из менеджера контекста:

In [ ]:
import tempfile
with tempfile.TemporaryFile() as handle:
    path = handle.name
    print(path)

In [ ]:
open(path)

### Примеры менеджеров контекста: syncronized

In [ ]:
import threading

class synchronized:
    def __init__(self):
        self.lock = threading.Lock()

    def __enter__(self):
        self.lock.acquire()

    def __exit__(self, *exc_info):
        self.lock.release()

with synchronized():
    do_something()

**Капитан сообщает:**

Большая часть примитивов синхронизации в Python, включая класс `Lock`, реализует протокол менеджера контекста.

Использовать менеджер `synchronized` не нужно — он интересен только в качестве примера.

### Примеры контекстных менеджеров: cd

In [18]:
import os

class cd:
    def __init__(self, path):
        self.path = path

    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)

    def __exit__(self, *exc_info):
        os.chdir(self.saved_cwd)

print(os.getcwd())

/home/ubuntu/projects/labs.in.ua/anaconda.dev


In [20]:
with cd("/tmp"):
    print(os.getcwd())
print(os.getcwd())

/tmp
/home/ubuntu/projects/labs.in.ua/anaconda.dev


### Менеджеры контекста: резюме

Менеджеры контекста — удобный способ управлять жизненным циклом ресурсов в Python.

Для работы с менеджером контекста используется оператор **`with`**.

Менеджером контекста является экземпляр любого класса, реализующего методы **`__enter__`** и **`__exit__`**.

Некоторые встроенные типы _(например, файлы и примитивы синхронизации)_ уже поддерживают протокол менеджеров контекста — этим можно и нужно пользоваться при написании кода.

## Модуль contextlib

### contextlib.closing

Менеджер контекста `closing` обобщает логику уже известного нам `opened` на экземпляр любого класса, реализующего метод `close`.

Реализовать `closing` самому несложно, но приятно, когда в стандартной библиотеке языка есть и такие мелочи.

С помощью `closing` можно, например, безопасно работать с HTTP ресурсами:


In [ ]:
from contextlib import closing
from urllib.request import urlopen

url = "http://slides.in.ua"

with closing(urlopen(url)) as page:
    do_something(page)

### contextlib.redirect_stdout

Менеджер контекста `redirect_stdout` позволяет локально перехватывать вывод в стандартный поток.

Пример использования:

In [ ]:
import io
from contextlib import redirect_stdout

handle = io.StringIO()

with redirect_stdout(handle):
    print("Hello, World!")

handle.getvalue()

**Ньюанс:** Если используется библиотека на C++, то её поток вывода не будет перенаправляться.

**Вопрос:** Как можно было бы реализовать `redirect_stdout`?

### contextlib.suppress

С помощью менеджера контекста `suppress` можно локально подавить исключения указанных типов:

In [26]:
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("example.txt")

Реализация менеджера не хитра:

In [27]:
class supress:
    def __init__(self, *suppressed):
        self.suppressed = suppressed

    def __enter__(self):
        pass

    def __exit__(self, exc_type, exc_value, tb):
        return exc_type is not None and issubclass(exc_type, suppressed)

При использовании `suppress`, как и в целом при работе с исключениями, стоит указывать наиболее специфичный тип исключения.

### contextlib.ContextDecorator

Базовый класс `ContextDecorator` позволяет объявлять менеджеры контекста, которые можно использовать как декораторы.

Зачем это нужно?

In [4]:
def f():
    with context():
        pass  # ...

In [ ]:
@context()
def f():
    pass  # ...

Переход к синтаксису декораторов:
* подчеркивает, что менеджер контекста применяется ко всему телу функции,
* позволяет сэкономить 4 пробела отступов :)

**Вопрос**: Как должен быть реализован менеджер контекста, чтобы его можно было использовать в качестве декоратора?

### Менеджеры контекста и декораторы

Для того, чтобы менеджер контекста можно было использовать как декоратор, достаточно унаследовать его от `ContextDecorator` _(примесь)_.

Модифицируем менеджер suppress из модуля contextlib, чтобы с помощью него можно было подавлять исключения во всей функции:

In [28]:
from contextlib import suppress, ContextDecorator

class suppressed(suppress, ContextDecorator):
    pass

@suppressed(IOError)
def do_something():
    pass

### contextlib.ExitStack

Что делать, если количество ресурсов может быть произвольным? Например:


In [ ]:
def merge_logs(output_path, *logs):
    handles = [open(log) for log in logs]
    
    with open(output_path, "wt") as output:
        merge(output, handles)
        
    for handle in handles:
        handle.close()

Правильный ответ: `ExitStack`.

Менеджер `ExitStack` позволяет управлять произвольным количеством менеджеров контекста:

In [29]:
from contextlib import ExitStack

def merge_logs(output_path, *logs):
    with ExitStack() as stack:
        handles = [stack.enter_context(open(log)) for log in logs]
        
        output = open(output_path, "wt")
        stack.enter_context(output)
        
        merge(output, handles)

### “Семантика” менеджера ExitStack

Менеджер `ExitStack` поддерживает стек вложенных менеджеров контекста:

In [ ]:
with ExitStack() as stack:
    stack.enter_context(resource_1)
    stack.enter_context(resource_2)
    do_something(resource_1, resource_2)

При выходе из контекста, `ExitStack` обходит список вложенных менеджеров контекста в обратном порядке и вызывает у каждого менеджера метод `__exit__`.

Менеджер `ExitStack` корректно обрабатывает ситуации,
* когда метод `__exit__` подавил исключение
* когда в процессе работы метода `__exit__` возникло новое исключение

### Модуль contextlib: резюме

Модуль `contextlib` содержит функции и классы, украшающие жизнь любителя менеджеров контекста.